In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

from sklearn.model_selection import train_test_split
from keras import backend as K

from keras.layers import Input, Dense, Lambda, Layer, Add, Multiply
from keras.models import Model, Sequential
from keras.datasets import mnist
%matplotlib inline

In [114]:
df = pd.read_csv("kdd_data.csv", parse_dates=['TIME_STAMP'])
df.drop(["id","MSG_ID","DSE_ID","COLL_ID","ENDPOINT_ID","ENDPOINT_URL","TIME_STAMP"],axis=1,inplace=True)
df.head()

,att1,att2,att3,att4,att5,attt6,label
0,0,1,7,0,257,818,0
1,0,1,7,0,311,354,0
2,4,2,2,0,89,41,0
3,0,1,7,0,192,491,0
4,0,1,7,0,192,755,0


In [115]:
close_avg_3 =  pd.DataFrame.rolling(df['attt6'], 3).mean()
close_avg_5 =  pd.DataFrame.rolling(df['attt6'], 5).mean()
close_avg_10 =  pd.DataFrame.rolling(df['attt6'], 10).mean()
df['close_avg_3']=close_avg_3
df['close_avg_5']=close_avg_5
df['close_avg_10']=close_avg_10


df= df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(how='any')

print(df.isnull().values.any())
df.head()

False


,att1,att2,att3,att4,att5,attt6,label,close_avg_3,close_avg_5,close_avg_10
9,0,1,7,0,309,1109,0,621.000000,926.2,709.0
10,0,1,7,0,279,891,0,808.000000,809.8,716.3
11,0,1,7,0,219,643,0,881.000000,679.4,745.2
12,0,1,7,0,296,6903,0,2812.333333,1994.0,1431.4
13,0,1,7,0,207,308,0,2618.000000,1970.8,1413.1


In [116]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["att5","attt6","att3","close_avg_3","close_avg_5","close_avg_10"]]=scaler.fit_transform(df[["att5","att3","attt6","close_avg_3","close_avg_5","close_avg_10"]])
df.head()

,att1,att2,att3,att4,att5,attt6,label,close_avg_3,close_avg_5,close_avg_10
9,0,1,0.008871,0,0.015669,0.5,0,0.013277,0.022535,0.016373
10,0,1,0.007127,0,0.014147,0.5,0,0.017274,0.018641,0.016834
11,0,1,0.005143,0,0.011105,0.5,0,0.018835,0.014278,0.018658
12,0,1,0.055217,0,0.015009,0.5,0,0.060126,0.058258,0.061969
13,0,1,0.002464,0,0.010496,0.5,0,0.055971,0.057482,0.060814


In [117]:
# train the VAE on MNIST digits
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.reshape(-1, original_dim) / 255.
# x_test = x_test.reshape(-1, original_dim) / 255.
# pd.DataFrame(y_test).head(5)


In [118]:
df_norm_x=df[['att1', 'att2', 'att3', 'att4', 'att5', 'attt6', 'close_avg_3',
      'close_avg_5', 'close_avg_10']]
#df_norm_x=df[['att1', 'att2', 'att3', 'att4', 'att5', 'attt6']]
df_norm_y=df['label']
x_train, x_test, y_train, y_test = train_test_split(df_norm_x, df_norm_y, test_size=0.20, random_state=26)


In [119]:
original_dim = 9
intermediate_dim = 4
latent_dim = 2
batch_size = 5
epochs = 50
epsilon_std = 1.0

In [120]:
def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)


class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs


In [121]:
decoder = Sequential([
    Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),
    Dense(original_dim, activation='sigmoid')
])

x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(x)

z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

eps = Input(tensor=K.random_normal(stddev=epsilon_std,
                                   shape=(K.shape(x)[0], latent_dim)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

x_pred = decoder(z)

vae = Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer='rmsprop', loss=nll)


y_test.shape

(199,)

In [122]:

vae.fit(x_train.values,
        x_train.values,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test.values, x_test.values))

encoder = Model(x, z_mu)


Train on 792 samples, validate on 199 samples
Epoch 1/50
792/792 [==============================] - 4s 5ms/step - loss: 26835360.5276 - val_loss: 5.5825
Epoch 2/50
792/792 [==============================] - 1s 1ms/step - loss: 7912623.1949 - val_loss: 4.9776
Epoch 3/50
792/792 [==============================] - 1s 1ms/step - loss: 2357604.2755 - val_loss: 4.1961
Epoch 4/50
792/792 [==============================] - 1s 1ms/step - loss: 709815.2028 - val_loss: 3.9353
Epoch 5/50
792/792 [==============================] - 1s 1ms/step - loss: 215959.9597 - val_loss: 3.3950
Epoch 6/50
792/792 [==============================] - 1s 1ms/step - loss: 66411.3242 - val_loss: 3.1146
Epoch 7/50
792/792 [==============================] - 1s 1ms/step - loss: 20442.5075 - val_loss: 3.0083
Epoch 8/50
792/792 [==============================] - 1s 1ms/step - loss: 11842.1907 - val_loss: 3.3697
Epoch 9/50
792/792 [==============================] - 1s 1ms/step - loss: 7738.1729 - val_loss: 2.9100
Epoch 10/5

In [123]:
# display a 2D plot of the digit classes in the latent space
z_test = encoder.predict(x_test, batch_size=batch_size)

In [124]:
z_test.shape

(199, 2)

In [125]:
x_decoded = decoder.predict(z_test)

mse = np.mean(np.power(x_test - x_decoded, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})
print(x_decoded.shape,mse.shape)

(199, 9) (199,)


In [126]:
d=pd.DataFrame(error_df)
thr_per=85
print(thr_per,"th percentile recon error", np.percentile(mse, thr_per))
d[d.reconstruction_error>np.percentile(mse, thr_per)]


85 th percentile recon error 0.08819514435055198


,reconstruction_error,true_class
542,0.118856,0
154,0.118007,0
260,0.092028,1
744,0.105067,1
720,0.091691,1
772,0.090820,1
191,0.089979,1
110,0.089614,1
53,0.092617,1
522,0.092447,1


In [19]:
dd=pd.concat([pd.DataFrame(y_test),d.reconstruction_error],axis=1)
dd.to_csv("da.csv")